### Pythonで綴る多変量解析　4-1. 因子分析(scikit-learn)
https://qiita.com/y_itoh/items/227cb33317ceb09199c2

因子分析は、多数ある変量の間に共通する因子を探り出して、それらの共通因子を使ってデータを評価するための技法です。

たとえば、マーケティングリサーチでは、ある質問の回答データ（多肢選択型でそれも選択肢が何十もある）について因子分析を行ない、ずっと少ない数の因子に置き換えます。\
それを今度はデータとして、クラスター分析（人分け）を行ない、つまりタイプ分類をしてプロファイリングをやったりします。\
各クラスターの性別・年齢別の構成比はどうなっているか、どのような特性をもった人たちなのか、市場全体に占めるボリュームは何％か、というようなことをやるわけです。

さて、12教科のテストの得点が1000人分、というダミーデータを因子分析用に作成しました。\
以下、scikit-learnを利用して、因子分析の大筋をつかみます。

In [ ]:
# 数値計算ライブラリ
import numpy as np
import pandas as pd
# 可視化ライブラリ
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
url = 'https://raw.githubusercontent.com/yumi-ito/sample_data/master/subject_scores.csv'
df = pd.read_csv(url)
df.head()

In [ ]:
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

#標本数(count)、平均値(mean)、標準偏差(std)、最小値(min)、四分位数(25%, 50%, 75%)、最大値(max)

In [ ]:
# matplotlibを日本語表示に対応させるモジュール
!pip install japanize-matplotlib
import japanize_matplotlib

# pandasのplottingメソッドをインポート
from pandas import plotting

# 散布図行列を表示
plotting.scatter_matrix(df, figsize=(12, 12), alpha=0.8)

In [ ]:
# sklearnの標準化モジュールをインポート
from sklearn.preprocessing import StandardScaler

# データを変換する計算式を生成
sc = StandardScaler()
sc.fit(df)

# 実際にデータを変換
z = sc.transform(df)

print(z)
print(z.shape)

In [ ]:
# sklearnのFactorAnalysis(因子分析)クラスをインポート
from sklearn.decomposition import FactorAnalysis as FA

# 因子数を指定
n_components=3

# 因子分析の実行
fa = FA(n_components, max_iter=5000) # モデルを定義
fitted = fa.fit_transform(z) # fitとtransformを一括処理

print(fitted)
print(fitted.shape)

In [ ]:
# 因子負荷量行列を取得
fa.components_.T

In [ ]:
# 変数Factor_loading_matrixに格納
Factor_loading_matrix = fa.components_.T

# データフレームに変換
pd.DataFrame(Factor_loading_matrix, 
             columns=["第1因子", "第2因子", "第3因子"], 
             index=[df.columns])

- 第１因子は、すべての変数がプラスになっているので、この因子が高くなると全変数が足並みをそろえて高くなります。そこで「総合学力」とか「基礎学力」などと命名できそうです
- 第２因子は、国語・英語の絶対値が大きく、いずれもプラスで、また倫理もプラスでやや高めとなっています。そこで「言語能力」と命名しておきましょう
- 第３因子は、数学・物理の絶対値がずば抜けて大きく、いずれもプラスになっているので、この因子は「数理能力」といえます

### Pythonで綴る多変量解析　4-2. 因子分析[因子数の推定]
https://qiita.com/y_itoh/items/ff57e7b02090fe900886

因子数をいくつにするか——。\
おおかた見当をつけて因子分析を何ケースか行ない、それらの結果を解釈してみて、もっとも妥当な因子数に決定しています。\
たとえば、6因子と見当をつけたら、4～6～8因子として5ケース行ない、そのうち分析軸としてもっとも使い勝手のよさそうな因子数を採用するという感じです。\
また、因子の解釈に際しては、次のような点に留意しています。
- 因子の解釈が一意的で
- 各因子がなるべく独立的であり
- 因子群が全体として網羅的である

さて、見当をつけるには、主成分分析によって求められる固有値が目安になります。\
ひとまず主成分分析のモデル生成までを行ない、その後で因子数を推定する3種類の方法をみていきます。

#### 主成分分析を行ない固有値を求める

In [ ]:
# 数値計算ライブラリ
import numpy as np
import pandas as pd
# 可視化ライブラリ
import matplotlib.pyplot as plt
%matplotlib inline
# 機械学習ライブラリ
import sklearn

In [ ]:
# matplotlibを日本語表記に対応
# !pip install japanize-matplotlib
# import japanize_matplotlib

In [ ]:
# URLを指定してcsvファイルを読み込む
url = 'https://raw.githubusercontent.com/yumi-ito/sample_data/master/subject_scores.csv'
df = pd.read_csv(url)

# 冒頭の5行分を表示
df.head()

In [ ]:
# sklearnの標準化メソッドをインポート
from sklearn.preprocessing import StandardScaler

# 標準化の実行
sc = StandardScaler() # インスタンスを生成･定義
sc.fit(df)            # データ変換の計算式を生成
z = sc.transform(df)  # 実際にデータを変換

print("標準化得点：\n", z)
print(" ")
print("行列の形状：", z.shape)

In [ ]:
# sklearnのPCA(主成分分析)クラスをインポート
from sklearn.decomposition import PCA

# 主成分分析のモデルを生成
pca = PCA() # インスタンスを生成･定義
pca.fit(z)  # 標準化得点データにもとづいてモデルを生成

#### 因子数の推定－固有値の値をもとにする場合

In [ ]:
# 固有値を取得
ev = pca.explained_variance_

# 行名･列名を付与してデータフレームに変換
pd.DataFrame(ev, 
             index=["PC{}".format(x + 1) for x in range(len(df.columns))], 
             columns=["固有値"])

#### 因子数の推定－スクリープロットをもとにする場合

In [ ]:
# 基準線(固有値1)をひくためのダミーデータ
ev_1 = np.ones(12)

# 変数を指定
plt.plot(ev, 's-')   # 主成分分析による固有値
plt.plot(ev_1, 's-') # ダミーデータ

# 軸名を指定
plt.xlabel("因子の数")
plt.ylabel("固有値")

plt.grid()
plt.show()

#### 因子数の推定－累積寄与率をもとにする場合

In [ ]:
# 寄与率の取得
evr = pca.explained_variance_ratio_

# 行名･列名を付与してデータフレームに変換
pd.DataFrame(evr, 
             index=["PC{}".format(x + 1) for x in range(len(df.columns))], 
             columns=["寄与率"])

In [ ]:
# 起点0と寄与率の累積値をプロット
plt.plot([0] + list(np.cumsum(evr)), "-o")

plt.xlabel("主成分の数")
plt.ylabel("累積寄与率")

plt.grid()
plt.show()

#### 因子数の推定に、なぜ主成分分析なのか？
因子分析と主成分分析とは、目的がまったく異なります。\
因子分析は、データに潜在する共通因子を見つけて、データの背後にある構造を知ろうとします。\
対して、主成分分析は、データのもつ情報を圧縮して次元数を減らし、データを取り扱いやすいように変換してくれます。\
ただし、そのやり方はよく似ています。

因子分析は、誤差がなるべく小さくなるような因子負荷量を見つけてきます。\
因子負荷量とは、共通因子が変数に及ぼしている影響度のことであり、また誤差とは、共通因子以外に各変数が独自にもっている因子のことです。

因子分析は、データを共通因子によって説明しようとするものです。\
ゆえに、独自因子は誤差として、その分だけ情報を失くすことになります。\
この情報の損失がなるべく小さくなるような因子負荷量を見つけてくれます。

次いで、主成分分析のパス図を示します。\
主成分負荷量は、変数が主成分に及ぼしている影響度です。\
主成分分析では、情報量＝分散として、情報の保持がなるべく大きくなるような主成分負荷量を見つけます。

### [統計]pythonで因子分析をおこなう
https://www.marsa-blog.com/2019/06/statisticsarticle1.html


- code pass: java-lang-programming/python_sample/statistics/factor_analysis/因子分析.ipynb\
https://github.com/java-lang-programming/python_sample/blob/master/statistics/factor_analysis/%E5%9B%A0%E5%AD%90%E5%88%86%E6%9E%90.ipynb

### Python scikit-learnによる因子分析
https://hk29.hatenablog.jp/entry/2018/06/09/010424

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn import datasets
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis as FA

In [ ]:
def my_load_dataset():  # データセットを読み込みXとYで返す
  boston = datasets.load_boston()
  boston_df=pd.DataFrame(boston.data) # 説明変数Xをpandas形式に変換
  boston_df.columns = boston.feature_names  # カラム名を挿入
  boston_df['PRICE'] = pd.DataFrame(boston.target)  # 目的変数Yをカラム名PRICEで追記   
  x_df = boston_df.drop("PRICE", axis=1)  # Xデータのみ抽出。pandas形式で列名有り
  #Y_df = boston_df.loc[:,['PRICE']]  # Yデータのみ抽出。pandas形式で列名有り
  y = boston_df.PRICE # Yデータのみ抽出。カラム名を指定して抽出のため、列名はない
  df_columns = boston.feature_names
  return x_df, y, df_columns  # XYデータをタプルで返す。[]を付けた場合はリストとなる。

In [ ]:
def my_scaler(X, df_columns): # 説明変数Xを標準化
  scaler = StandardScaler()
  x_sc = scaler.fit_transform(X)
  ### plot    
  # sklearnの上記のようなfit関連の実行後、データの書式がnumpy形式となりカラム(列)名が無くなる。
  # そのため、再度、pandas形式に変換する(次の2行)。これはグラフの軸名で使用するため。
  x_sc = pd.DataFrame(x_sc)
  x_sc.columns = df_columns
  return x_sc

In [ ]:
def my_fa(X_sc, Y, n): # 因子分析
  fa = FA(n_components=n, max_iter=500)
  fa.fit(X_sc)
  print("fa.components_")
  print(fa.components_)
    
  x_fa = fa.fit_transform(X_sc)
  ### plot
  x_data = x_fa[:,0]
  y_data = x_fa[:,1]
  fa0 = fa.components_[0]
  fa1 = fa.components_[1]
  plt.figure(figsize=(9,9))
  plt.scatter(x_data, y_data, c=Y/len(set(Y)), marker=".")
  for i in range(fa0.shape[0]):
    plt.arrow(0, 0, fa0[i]*1.5, fa1[i]*1.5, color='r', width=0.0005, head_width=0.07, alpha=0.8)
    plt.text(fa0[i]*1.8, fa1[i]*1.8, X_sc.columns.values[i], color='r', fontsize = 18)
  plt.xlim([-2.5,2.5])
  plt.ylim([-2.5,2.5])
  plt.xlabel('Factor 1', fontsize = 20)
  plt.ylabel('Factor 2', fontsize = 20)
  plt.title(str(n)+" factor", fontsize = 20)
  plt.tick_params(labelsize = 18)
  plt.grid()
  c = patches.Circle(xy=(0, 0), radius=1, ec='r', fill=False)
  plt.axes().add_patch(c)
  #plt.show()
  plt.savefig("FA_" + str('%02g' %n) + '.png')

In [ ]:
def my_regression(X, Y, mytitle='Ref', mylabel='Multiple regression analysis'): # 線形重回帰分析
  clf = linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
  clf.fit(X, Y)
  print(pd.DataFrame({"Name":X.columns, "Coefficients":clf.coef_}).sort_values(by='Coefficients'))  # 係数でソート
  print(clf.intercept_) # 切片
  print(clf.score(X,Y)) # R^2

  plt.figure()
  plt.title(mytitle)
  plt.xlabel('RM (number of rooms)', fontsize=14)
  plt.ylabel('PRICE (target)', fontsize=14)
  plt.scatter(X["RM"], Y, c='blue', label='Raw data')
  plt.scatter(X.RM, clf.predict(X), c='red', label=mylabel)
  plt.legend(loc='lower right', fontsize=12)
  plt.show()

In [ ]:
if __name__ == '__main__':
  # データセットを読み込み次のの3つを返す。X, Y, 説明変数Xのカラム(列)名
  data = my_load_dataset()
  X = data[0]
  Y = data[1]
  df_columns = data[2]
  print(X)
    
  # 説明変数Xを標準化
  X_sc = my_scaler(X, df_columns)
  print(X_sc)
    
  # 因子分析
  mycheck_Num=[2,3,12,13]
  for i in mycheck_Num:
    print(i)
    my_fa(X_sc, Y, i)
    
  # 説明変数Xを減らす
  # 因子分析の結果から独自判断で下記4つの説明変数を削除
  drop_col_1 = ['LSTAT', 'INDUS', 'NOX', 'AGE']
  X1 = X.drop(drop_col_1, axis=1) # X1は9つの説明変数Xで構成されるデータフレーム(pandas形式)

  # 線形重回帰分析
  my_regression(X, Y, mytitle='Ref', mylabel='13feature analysis')  # 1.説明変数X全13因子を使用しての回帰
  my_regression(X1, Y, mytitle='After FA', mylabel='9feature analysis') # 2.説明変数X9因子を使用しての回帰

参考までに、主成分分析と因子分析の違いを記載する。
  
主成分分析の場合は、目的変数Yに対する説明変数を主成分  
$
PC_i ＝ X_1\times e_1 + X_2 \times e_2 + \cdots
$  
と呼ぶ合成変数に置換している。
- その定義は分散に着目しており、第一主成分$PC_1$が最大の分散方向に軸をとった成分で、最も寄与率が高いため、分析時のコンポーネント数が2だろうが、10だろうが$PC_1$の結果に違いはない
  
一方、因子分析の場合は、式  
$
X_i = Factor_1 \times Load_1 + Factor_2 \times Load_2 + \cdots + e_i
$  
をみるとわかる通り共通因子$Factor$は、説明変数$X_i$に従属しており、目的変数$Y$に対する説明変数は$X_i$のままである。  
そのため、共通因子$Factor$の数に依って負荷量$Load$の感度(大きさ)は異なる。
- 別の表現をすると、その数に依存して相対関係に変化は生じず、同じ解釈に成りうる
- 結局は、人の判断に依る
  
データ分析手法は色々あるが、最終目的は回帰式の作成と利用であろうから、個人的にはいきなり重回帰分析、もしくはランダムフォレストで良いと思う。